In [59]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np

In [69]:
def adjust_minutes(dt):
    if dt.minute <= 10:
        return dt.replace(minute=0)
    else:
        return dt

df_pv=pd.read_csv("pv_inverter_new.csv",sep=";") # read file
df_pv['date_time'] = pd.to_datetime(df_pv['gather_time']).dt.floor('min')
df_pv['date_time'] = df_pv['date_time'].apply(adjust_minutes)
specific_times = [pd.Timestamp('today').replace(hour=h, minute=0, second=0, microsecond=0).time() for h in [2, 5, 8, 11, 14, 17, 20, 23]]
df_pv = df_pv[df_pv['date_time'].dt.time.isin(specific_times)]
expected_sequence = [6, 7, 8, 9, 10]
pattern_index = pd.Series(range(len(df_pv))).mod(len(expected_sequence))
matches_sequence = (df_pv['device_id'].reset_index(drop=True).values == pd.Series(expected_sequence)[pattern_index].values).all()

matches_sequence

True

In [70]:
df_pv = df_pv.copy()
df_pv['difference'] = df_pv['date_time'] - df_pv['date_time'].shift(5)
df_pv[df_pv['difference'] != pd.Timedelta('0 days 03:00:00')]

,id,device_id,gather_time,active_power,direct_power,charge_capacity,date_time,difference
10,104953,6,2024-02-23 02:10:00,0.0,0.0,0.0,2024-02-23 02:00:00,NaT
11,104954,7,2024-02-23 02:10:00,0.0,0.0,0.0,2024-02-23 02:00:00,NaT
12,104955,8,2024-02-23 02:10:00,0.0,0.0,0.0,2024-02-23 02:00:00,NaT
13,104956,9,2024-02-23 02:10:00,0.0,0.0,0.0,2024-02-23 02:00:00,NaT
14,104957,10,2024-02-23 02:10:00,0.0,0.0,0.0,2024-02-23 02:00:00,NaT


In [71]:
df_pv

,id,device_id,gather_time,active_power,direct_power,charge_capacity,date_time,difference
10,104953,6,2024-02-23 02:10:00,0.00,0.00,0.0,2024-02-23 02:00:00,NaT
11,104954,7,2024-02-23 02:10:00,0.00,0.00,0.0,2024-02-23 02:00:00,NaT
12,104955,8,2024-02-23 02:10:00,0.00,0.00,0.0,2024-02-23 02:00:00,NaT
13,104956,9,2024-02-23 02:10:00,0.00,0.00,0.0,2024-02-23 02:00:00,NaT
14,104957,10,2024-02-23 02:10:00,0.00,0.00,0.0,2024-02-23 02:00:00,NaT
...,...,...,...,...,...,...,...,...
1240,106183,6,2024-03-04 08:10:01,3.70,4.24,0.9,2024-03-04 08:00:00,0 days 03:00:00
1241,106184,7,2024-03-04 08:10:01,3.80,3.99,0.9,2024-03-04 08:00:00,0 days 03:00:00
1242,106185,8,2024-03-04 08:10:01,3.80,4.19,0.9,2024-03-04 08:00:00,0 days 03:00:00
1243,106186,9,2024-03-04 08:10:01,4.02,4.39,0.8,2024-03-04 08:00:00,0 days 03:00:00


In [72]:
df_pv.to_csv("df_pv.csv")

In [73]:
df_weather=pd.read_excel("58752.23.02.2024.04.03.2024.1.0.0.cn.utf8.00000000.xls",skiprows=6)
df_weather=df_weather.iloc[::-1].reset_index(drop=True)
df_weather.drop(['ff10', 'N', 'Cl', 'Nh', 'Cm', 'Ch', 'E', 'Tg', "E'", 'sss', 'H', 'P', 'Pa',
                 'W1', 'W2', 'Tn', 'Tx', 'tR', 'ff3', 'WW', 'VV', 'Td'], axis=1, inplace=True)
df_weather['date_time'] = pd.to_datetime(df_weather['当地时间 温州市'], dayfirst=True).dt.floor('min')
df_weather.drop(['当地时间 温州市'], axis=1, inplace=True)
pd.set_option('future.no_silent_downcasting', True)
df_weather['RRR']=df_weather['RRR'].replace('无降水', 0)
def DD_to_N(dd):
    direction={
        '北':'0/1/',
        '东':'1/0/',
        '南':'0/2/',
        '西':'2/0/'
    }
    if isinstance(dd, str):
        for char, num in direction.items():
            dd = dd.replace(char, num)
        numbers = re.findall('\d+', dd)[:4]
        numbers = [int(num) for num in numbers]
        numbers = [-1 if x == 2 else x for x in numbers]
        new_numbers = [numbers[i:i + 2] for i in range(0, len(numbers), 2)]
        if len(new_numbers)==0:
            result=[0,0]
        else:
            result=[0,0]
            for i in range(len(new_numbers)):
                result=[a + b for a, b in zip(result, new_numbers[i])]
    else:
        result=[0,0]
    return result
df_weather[['DD_WE', 'DD_NS']] = df_weather['DD'].apply(DD_to_N).apply(pd.Series)
df_weather.drop('DD', axis=1, inplace=True)
df_weather.loc[:, 'Po(p)'] = (df_weather['Po'] * 1.322).round(2)
df_weather.drop('Po', axis=1, inplace=True)

from pvlib import location
from pvlib import irradiance
tz='Asia/Shanghai'
lat, lon = 27.962847, 120.736522
site = location.Location(lat, lon, tz=tz)

def get_irradiance(site_location, start, end, tilt, surface_azimuth):
    # print(site_location)
    times = pd.date_range(start=start, end=end, freq='3h', tz=site_location.tz)
    clearsky = site_location.get_clearsky(times)
    # print(clearsky)
    solar_position = site_location.get_solarposition(times=times)
    POA_irradiance = irradiance.get_total_irradiance(
        surface_tilt=tilt,
        surface_azimuth=surface_azimuth,
        dni=clearsky['dni'],
        ghi=clearsky['ghi'],
        dhi=clearsky['dhi'],
        solar_zenith=solar_position['apparent_zenith'],
        solar_azimuth=solar_position['azimuth'])

    return pd.DataFrame({'date_time': times,'POA': POA_irradiance['poa_global']})

#获取辐照度数据，假设倾斜25度，和朝南的阵列
irradiance = get_irradiance(site, '2024-02-23 02:00:00', '2024-03-04 08:00:00', 25, 180)
irradiance['date_time'] = irradiance['date_time'].dt.tz_localize(None)
irradiance.reset_index(inplace=True)
irradiance.drop('index',axis=1,inplace=True)
df_weather = pd.merge(df_weather, irradiance, on='date_time', how='left')


df_weather['difference'] = df_weather['date_time'] - df_weather['date_time'].shift(1)
df_weather[df_weather['difference'] != pd.Timedelta('0 days 03:00:00')]

Location: 
  name: None
  latitude: 27.962847
  longitude: 120.736522
  altitude: 0
  tz: Asia/Shanghai
                                  ghi         dni        dhi
2024-02-23 02:00:00+08:00    0.000000    0.000000   0.000000
2024-02-23 05:00:00+08:00    0.000000    0.000000   0.000000
2024-02-23 08:00:00+08:00  277.059612  707.282992  51.847290
2024-02-23 11:00:00+08:00  785.466605  935.526047  86.418907
2024-02-23 14:00:00+08:00  718.170567  919.733732  82.531290
...                               ...         ...        ...
2024-03-03 20:00:00+08:00    0.000000    0.000000   0.000000
2024-03-03 23:00:00+08:00    0.000000    0.000000   0.000000
2024-03-04 02:00:00+08:00    0.000000    0.000000   0.000000
2024-03-04 05:00:00+08:00    0.000000    0.000000   0.000000
2024-03-04 08:00:00+08:00  314.071539  700.868137  62.739077

[83 rows x 3 columns]


,T,U,Ff,RRR,date_time,DD_WE,DD_NS,Po(p),POA,difference
0,5.8,93,1,3,2024-02-23 02:00:00,0,1,1013.71,0.0,NaT


In [74]:
df_weather

,T,U,Ff,RRR,date_time,DD_WE,DD_NS,Po(p),POA,difference
0,5.8,93,1,3,2024-02-23 02:00:00,0,1,1013.71,0.000000,NaT
1,5.8,86,2,3,2024-02-23 05:00:00,1,1,1013.05,0.000000,0 days 03:00:00
2,5.2,95,1,17,2024-02-23 08:00:00,0,1,1014.90,366.468452,0 days 03:00:00
3,6.6,94,1,20,2024-02-23 11:00:00,1,1,1015.43,959.975349,0 days 03:00:00
4,8.2,77,2,20,2024-02-23 14:00:00,0,1,1013.84,882.118722,0 days 03:00:00
...,...,...,...,...,...,...,...,...,...,...
80,9.5,74,1,0,2024-03-04 02:00:00,-1,0,1006.31,0.000000,0 days 03:00:00
81,9.6,76,2,0,2024-03-04 05:00:00,-1,0,1004.72,0.000000,0 days 03:00:00
82,9.6,88,3,0.5,2024-03-04 08:00:00,-1,0,1004.85,384.516701,0 days 03:00:00
83,10.0,94,3,4,2024-03-04 11:00:00,-1,0,1003.13,NaN,0 days 03:00:00


In [75]:
df_merged = pd.merge(df_pv, df_weather, on='date_time', how='left')
df_merged.drop(['difference_x','difference_y', 'gather_time'], axis=1, inplace=True)
df_merged = df_merged[~df_merged['date_time'].dt.hour.isin([2, 23])]
df_merged

,id,device_id,active_power,direct_power,charge_capacity,date_time,T,U,Ff,RRR,DD_WE,DD_NS,Po(p),POA
5,104968,6,0.00,0.00,0.0,2024-02-23 05:00:00,5.8,86,2,3,1,1,1013.05,0.000000
6,104969,7,0.00,0.00,0.0,2024-02-23 05:00:00,5.8,86,2,3,1,1,1013.05,0.000000
7,104970,8,0.00,0.00,0.0,2024-02-23 05:00:00,5.8,86,2,3,1,1,1013.05,0.000000
8,104971,9,0.00,0.00,0.0,2024-02-23 05:00:00,5.8,86,2,3,1,1,1013.05,0.000000
9,104972,10,0.00,0.00,0.0,2024-02-23 05:00:00,5.8,86,2,3,1,1,1013.05,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,106183,6,3.70,4.24,0.9,2024-03-04 08:00:00,9.6,88,3,0.5,-1,0,1004.85,384.516701
411,106184,7,3.80,3.99,0.9,2024-03-04 08:00:00,9.6,88,3,0.5,-1,0,1004.85,384.516701
412,106185,8,3.80,4.19,0.9,2024-03-04 08:00:00,9.6,88,3,0.5,-1,0,1004.85,384.516701
413,106186,9,4.02,4.39,0.8,2024-03-04 08:00:00,9.6,88,3,0.5,-1,0,1004.85,384.516701


In [76]:
df_merged.to_csv('df_merged.csv')

In [83]:
df=df_merged
df['cc_diff'] = df['charge_capacity'] - df['charge_capacity'].shift(5)
df['RRR'] = pd.to_numeric(df['RRR'], errors='coerce')
df['POA'] = df['POA'].round(1)
columns = ['active_power', 'date_time', 'T', 'U', 'Ff', 'RRR', 'DD_WE', 'DD_NS', 'Po(p)', 'POA']
for col in columns:
    df[f'{col}_b'] = df[col].shift(5)

df['difference'] = df['date_time'] - df['date_time'].shift(5)
df = df[df['difference'] == pd.Timedelta(hours=3)]
df=df.copy()
df['cc_diff'] = df['cc_diff'].round(1)
df['hour'] = df['date_time'].dt.hour 
df

,id,device_id,active_power,direct_power,charge_capacity,date_time,T,U,Ff,RRR,...,T_b,U_b,Ff_b,RRR_b,DD_WE_b,DD_NS_b,Po(p)_b,POA_b,difference,hour
10,104983,6,2.31,2.69,1.8,2024-02-23 08:00:00,5.2,95,1,17.0,...,5.8,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8
11,104984,7,2.38,2.79,1.9,2024-02-23 08:00:00,5.2,95,1,17.0,...,5.8,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8
12,104985,8,2.37,2.83,1.9,2024-02-23 08:00:00,5.2,95,1,17.0,...,5.8,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8
13,104986,9,2.44,2.84,2.2,2024-02-23 08:00:00,5.2,95,1,17.0,...,5.8,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8
14,104987,10,2.32,2.68,1.8,2024-02-23 08:00:00,5.2,95,1,17.0,...,5.8,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,106183,6,3.70,4.24,0.9,2024-03-04 08:00:00,9.6,88,3,0.5,...,9.6,76.0,2.0,0.0,-1.0,0.0,1004.72,0.0,0 days 03:00:00,8
411,106184,7,3.80,3.99,0.9,2024-03-04 08:00:00,9.6,88,3,0.5,...,9.6,76.0,2.0,0.0,-1.0,0.0,1004.72,0.0,0 days 03:00:00,8
412,106185,8,3.80,4.19,0.9,2024-03-04 08:00:00,9.6,88,3,0.5,...,9.6,76.0,2.0,0.0,-1.0,0.0,1004.72,0.0,0 days 03:00:00,8
413,106186,9,4.02,4.39,0.8,2024-03-04 08:00:00,9.6,88,3,0.5,...,9.6,76.0,2.0,0.0,-1.0,0.0,1004.72,0.0,0 days 03:00:00,8


In [84]:
df.to_csv('df_cc.csv')

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 255 entries, 10 to 414
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   id               255 non-null    int64          
 1   device_id        255 non-null    int64          
 2   active_power     255 non-null    float64        
 3   direct_power     255 non-null    float64        
 4   charge_capacity  255 non-null    float64        
 5   date_time        255 non-null    datetime64[ns] 
 6   T                255 non-null    float64        
 7   U                255 non-null    int64          
 8   Ff               255 non-null    int64          
 9   RRR              255 non-null    float64        
 10  DD_WE            255 non-null    int64          
 11  DD_NS            255 non-null    int64          
 12  Po(p)            255 non-null    float64        
 13  POA              255 non-null    float64        
 14  cc_diff          255 non-null 